In [1]:
import os
os.chdir("../")
%pwd

'e:\\Deep Learning\\TENSORFLOW\\rice_image_detection'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class ModelPreparationConfig:
    root_dir: Path
    model_dir: Path
    weight_decay: float
    classes: int
    input_image_size: list

In [3]:
from src.RICE_IMAGE_DETECTION.constants import *
from src.RICE_IMAGE_DETECTION.utils.common import read_yaml, create_directories

In [4]:
class ConfigureationManager:
    def __init__(self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_preparation_config(self) -> ModelPreparationConfig:
        config = self.config.prepare_model

        create_directories([config.root_dir])

        model_preparation_config = ModelPreparationConfig(
            root_dir = Path(config.root_dir),
            model_dir = Path(config.model_dir),
            weight_decay = self.params.weight_decay,
            classes = self.params.classes,
            input_image_size= self.params.input_image_size
        )

        return model_preparation_config

In [5]:
import os
import tensorflow as tf
from tensorflow import keras
from src.RICE_IMAGE_DETECTION import logger

In [6]:
class ModelPreparation:
    def __init__(self, config: ModelPreparationConfig):
        self.config = config

    def model(self):
        logger.info(f"------------Model Preparation Started-------------")
        classes = self.config.classes
        weight_decay = self.config.weight_decay
        input_shape = self.config.input_image_size
        cnn = tf.keras.models.Sequential([
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape,
                                kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
            tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                                kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
            tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=128, activation='relu',
                                kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
            tf.keras.layers.Dense(classes, activation='softmax')
            ])
        logger.info(f"------------Model Preparation Ended-------------")
        return cnn
    
    def save_model(self, model):
        model_dir = Path(self.config.model_dir)
        model.save(model_dir)
        logger.info(f"---MODEL SAVED------")


In [7]:
try:
    config = ConfigureationManager()
    model_preparation_config = config.get_model_preparation_config()
    model_preparation = ModelPreparation(config=model_preparation_config)
    cnn = model_preparation.model()
    model_preparation.save_model(cnn)
    logger.info(f"---MODEL SUMMARY------")
    logger.info(cnn.summary())
except Exception as e:
    raise e

[2024-06-30 14:55:24,221: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-30 14:55:24,225: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-30 14:55:24,225: INFO: common: created directory at: artifacts]
[2024-06-30 14:55:24,225: INFO: common: created directory at: artifacts/prepare_model]
[2024-06-30 14:55:24,225: INFO: 483240577: ------------Model Preparation Started-------------]
[2024-06-30 14:55:24,502: INFO: 483240577: ------------Model Preparation Ended-------------]
[2024-06-30 14:55:24,502: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
[2024-06-30 14:55:24,720: INFO: 483240577: ---MODEL SAVED------]
[2024-06-30 14:55:24,721: INFO: 374100628: ---MODEL SUMMARY------]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Par